# Teoria da Produção - Funções de Produção e Custos

Este notebook demonstra como usar a biblioteca para analisar funções de produção, isoquantas, minimização de custos e otimização da produção.

In [ ]:
# Importações necessárias
import sys
sys.path.append('/Workspace/Repos/your-repo-name')

import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from microeconomics.production import (
    cobb_douglas_production, cost_minimization, returns_to_scale,
    ProductionFunction
)

## 1. Função de Produção Cobb-Douglas

In [ ]:
# Criar função de produção Cobb-Douglas: Q = 2 * L^0.7 * K^0.3
production_cd = cobb_douglas_production(
    total_factor_productivity=2,
    labor_elasticity=0.7,
    capital_elasticity=0.3
)

# Calcular produção para diferentes combinações de trabalho e capital
labor_vals = [1, 2, 3, 4, 5]
capital_vals = [1, 2, 3, 4, 5]

print("Função de Produção Cobb-Douglas: Q = 2 * L^0.7 * K^0.3")
print("\nProdução para diferentes combinações de L e K:")
print("L\tK\tQ\tMPL\tMPK")
print("-" * 40)

for L in labor_vals:
    for K in capital_vals[:3]:  # Limitar para não sobrecarregar
        Q = production_cd.output(L, K)
        MPL = production_cd.marginal_product_labor(L, K)
        MPK = production_cd.marginal_product_capital(L, K)
        print(f"{L}\t{K}\t{Q:.2f}\t{MPL:.2f}\t{MPK:.2f}")

## 2. Isoquantas - Curvas de Produção Constante

In [ ]:
# Plotar isoquantas para diferentes níveis de produção
output_levels = [5, 10, 15, 20, 25]
fig = production_cd.plot_isoquants(
    output_levels,
    labor_range=(0.5, 15),
    title="Isoquantas - Cobb-Douglas"
)
fig.show()

## 3. Análise de Produtividade Marginal

In [ ]:
# Analisar como a produtividade marginal varia com os insumos
labor_range = np.linspace(1, 10, 20)
capital_fixed = 5

# Calcular produto total, marginal e médio do trabalho
total_product = [production_cd.output(L, capital_fixed) for L in labor_range]
marginal_product = [production_cd.marginal_product_labor(L, capital_fixed) for L in labor_range]
average_product = [tp/L for tp, L in zip(total_product, labor_range)]

# Criar gráfico
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=labor_range, y=total_product,
    mode='lines+markers', name='Produto Total',
    line=dict(color='blue')
))

fig.add_trace(go.Scatter(
    x=labor_range, y=marginal_product,
    mode='lines+markers', name='Produto Marginal',
    line=dict(color='red')
))

fig.add_trace(go.Scatter(
    x=labor_range, y=average_product,
    mode='lines+markers', name='Produto Médio',
    line=dict(color='green')
))

fig.update_layout(
    title=f'Produtos Total, Marginal e Médio do Trabalho (K = {capital_fixed})',
    xaxis_title='Trabalho (L)',
    yaxis_title='Produto',
    template='plotly_white'
)

fig.show()

print(f"Com capital fixo em {capital_fixed} unidades:")
print(f"Máximo produto marginal do trabalho: {max(marginal_product):.2f}")
print(f"Máximo produto médio do trabalho: {max(average_product):.2f}")

## 4. Minimização de Custos

In [ ]:
# Problema de minimização de custos
target_output = 20  # Meta de produção
wage_rate = 10      # Salário por unidade de trabalho
rental_rate = 15    # Taxa de aluguel do capital por unidade

# Encontrar combinação ótima de insumos
L_optimal, K_optimal, min_cost = cost_minimization(
    production_cd, target_output, wage_rate, rental_rate
)

print("=== PROBLEMA DE MINIMIZAÇÃO DE CUSTOS ===")
print(f"Meta de Produção: {target_output} unidades")
print(f"Salário (w): R$ {wage_rate} por unidade de trabalho")
print(f"Taxa de Aluguel do Capital (r): R$ {rental_rate} por unidade")
print()
print("SOLUÇÃO ÓTIMA:")
print(f"Trabalho ótimo: {L_optimal:.2f} unidades")
print(f"Capital ótimo: {K_optimal:.2f} unidades")
print(f"Custo mínimo: R$ {min_cost:.2f}")
print()

# Verificar se a produção atinge a meta
actual_output = production_cd.output(L_optimal, K_optimal)
print(f"Produção real com inputs ótimos: {actual_output:.2f}")
print(f"Diferença da meta: {abs(actual_output - target_output):.6f}")

## 5. Análise de Retornos de Escala

In [ ]:
# Analisar retornos de escala
scale_analysis = returns_to_scale(production_cd)
print(f"Retornos de Escala: {scale_analysis}")

# Verificação numérica
L_base = 2
K_base = 3
base_output = production_cd.output(L_base, K_base)

scales = [0.5, 1.0, 1.5, 2.0, 2.5, 3.0]
results = []

for scale in scales:
    L_scaled = scale * L_base
    K_scaled = scale * K_base
    scaled_output = production_cd.output(L_scaled, K_scaled)
    output_ratio = scaled_output / base_output
    
    results.append({
        'scale': scale,
        'labor': L_scaled,
        'capital': K_scaled,
        'output': scaled_output,
        'output_ratio': output_ratio,
        'returns_type': 'Crescente' if output_ratio > scale else 
                       'Constante' if abs(output_ratio - scale) < 0.01 else 'Decrescente'
    })

returns_df = pd.DataFrame(results)
display(returns_df)

# Plotar análise de retornos de escala
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=returns_df['scale'], y=returns_df['output_ratio'],
    mode='lines+markers', name='Razão de Produto Observada',
    line=dict(color='blue')
))

fig.add_trace(go.Scatter(
    x=returns_df['scale'], y=returns_df['scale'],
    mode='lines', name='Retornos Constantes (referência)',
    line=dict(color='red', dash='dash')
))

fig.update_layout(
    title='Análise de Retornos de Escala',
    xaxis_title='Escala dos Insumos',
    yaxis_title='Razão do Produto',
    template='plotly_white'
)

fig.show()

## 6. Comparação de Diferentes Funções de Produção

In [ ]:
# Criar diferentes tipos de funções de produção
prod_cd = cobb_douglas_production(total_factor_productivity=2, labor_elasticity=0.7, capital_elasticity=0.3)
prod_linear = ProductionFunction('linear', labor_productivity=2, capital_productivity=1.5)
prod_leontief = ProductionFunction('leontief', labor_requirement=0.5, capital_requirement=0.3)

# Comparar produção para diferentes combinações de insumos
labor_vals = np.linspace(1, 8, 15)
capital_fixed = 4

output_cd = [prod_cd.output(L, capital_fixed) for L in labor_vals]
output_linear = [prod_linear.output(L, capital_fixed) for L in labor_vals]
output_leontief = [prod_leontief.output(L, capital_fixed) for L in labor_vals]

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=labor_vals, y=output_cd,
    mode='lines+markers', name='Cobb-Douglas',
    line=dict(color='blue')
))

fig.add_trace(go.Scatter(
    x=labor_vals, y=output_linear,
    mode='lines+markers', name='Linear',
    line=dict(color='red')
))

fig.add_trace(go.Scatter(
    x=labor_vals, y=output_leontief,
    mode='lines+markers', name='Leontief',
    line=dict(color='green')
))

fig.update_layout(
    title=f'Comparação de Funções de Produção (K = {capital_fixed})',
    xaxis_title='Trabalho (L)',
    yaxis_title='Produto (Q)',
    template='plotly_white'
)

fig.show()

## 7. Análise de Custos em Diferentes Níveis de Produção

In [ ]:
# Analisar custos para diferentes níveis de produção
output_targets = np.linspace(5, 30, 15)
wage = 8
rental = 12

cost_results = []
for target in output_targets:
    L_opt, K_opt, cost_min = cost_minimization(production_cd, target, wage, rental)
    
    # Calcular custos médio e marginal
    avg_cost = cost_min / target
    
    cost_results.append({
        'output': target,
        'labor': L_opt,
        'capital': K_opt,
        'total_cost': cost_min,
        'average_cost': avg_cost
    })

cost_df = pd.DataFrame(cost_results)

# Calcular custo marginal (aproximação numérica)
cost_df['marginal_cost'] = cost_df['total_cost'].diff() / cost_df['output'].diff()

# Plotar curvas de custo
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=['Custo Total', 'Custo Médio', 'Custo Marginal', 'Insumos Ótimos']
)

# Custo Total
fig.add_trace(go.Scatter(
    x=cost_df['output'], y=cost_df['total_cost'],
    mode='lines+markers', name='Custo Total'
), row=1, col=1)

# Custo Médio
fig.add_trace(go.Scatter(
    x=cost_df['output'], y=cost_df['average_cost'],
    mode='lines+markers', name='Custo Médio'
), row=1, col=2)

# Custo Marginal
fig.add_trace(go.Scatter(
    x=cost_df['output'][1:], y=cost_df['marginal_cost'][1:],
    mode='lines+markers', name='Custo Marginal'
), row=2, col=1)

# Insumos ótimos
fig.add_trace(go.Scatter(
    x=cost_df['output'], y=cost_df['labor'],
    mode='lines+markers', name='Trabalho'
), row=2, col=2)

fig.add_trace(go.Scatter(
    x=cost_df['output'], y=cost_df['capital'],
    mode='lines+markers', name='Capital'
), row=2, col=2)

fig.update_layout(height=600, title_text="Análise de Custos")
fig.show()

display(cost_df.head(10))

## 8. Análise de Eficiência Técnica

In [ ]:
# Analisar eficiência técnica comparando diferentes empresas
np.random.seed(42)

# Simular dados de diferentes empresas
n_firms = 20
firms_data = []

for i in range(n_firms):
    # Inputs observados
    labor = np.random.uniform(2, 8)
    capital = np.random.uniform(1, 6)
    
    # Output máximo teórico (fronteira de produção)
    max_output = production_cd.output(labor, capital)
    
    # Output observado (com ineficiência aleatória)
    efficiency = np.random.uniform(0.7, 1.0)  # Eficiência entre 70% e 100%
    actual_output = max_output * efficiency
    
    firms_data.append({
        'firm_id': i+1,
        'labor': labor,
        'capital': capital,
        'actual_output': actual_output,
        'max_output': max_output,
        'efficiency': efficiency
    })

firms_df = pd.DataFrame(firms_data)

# Plotar análise de eficiência
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=['Distribuição de Eficiência', 'Fronteira de Produção']
)

# Histograma de eficiência
fig.add_trace(go.Histogram(
    x=firms_df['efficiency'],
    nbinsx=10,
    name='Eficiência'
), row=1, col=1)

# Scatter plot: output observado vs máximo
fig.add_trace(go.Scatter(
    x=firms_df['max_output'],
    y=firms_df['actual_output'],
    mode='markers',
    name='Empresas',
    text=[f'Empresa {i}' for i in firms_df['firm_id']],
    hovertemplate='<b>%{text}</b><br>Output Máximo: %{x:.2f}<br>Output Real: %{y:.2f}<extra></extra>'
), row=1, col=2)

# Linha de 45 graus (eficiência 100%)
max_val = firms_df['max_output'].max()
fig.add_trace(go.Scatter(
    x=[0, max_val],
    y=[0, max_val],
    mode='lines',
    name='Fronteira (100% eficiência)',
    line=dict(color='red', dash='dash')
), row=1, col=2)

fig.update_layout(height=500, title_text="Análise de Eficiência Técnica")
fig.show()

# Estatísticas de eficiência
print("=== ANÁLISE DE EFICIÊNCIA TÉCNICA ===")
print(f"Eficiência média: {firms_df['efficiency'].mean():.3f}")
print(f"Eficiência mediana: {firms_df['efficiency'].median():.3f}")
print(f"Eficiência mínima: {firms_df['efficiency'].min():.3f}")
print(f"Eficiência máxima: {firms_df['efficiency'].max():.3f}")
print(f"Desvio padrão: {firms_df['efficiency'].std():.3f}")

# Empresas mais e menos eficientes
most_efficient = firms_df.loc[firms_df['efficiency'].idxmax()]
least_efficient = firms_df.loc[firms_df['efficiency'].idxmin()]

print(f"\nEmpresa mais eficiente: {most_efficient['firm_id']} ({most_efficient['efficiency']:.3f})")
print(f"Empresa menos eficiente: {least_efficient['firm_id']} ({least_efficient['efficiency']:.3f})")

display(firms_df.sort_values('efficiency', ascending=False).head())

## 9. Resumo e Conclusões

In [ ]:
print("=== RESUMO DA ANÁLISE DE TEORIA DA PRODUÇÃO ===")
print()
print("1. FUNÇÃO DE PRODUÇÃO ANALISADA:")
A = production_cd.parameters.get('total_factor_productivity')
alpha = production_cd.parameters.get('labor_elasticity')
beta = production_cd.parameters.get('capital_elasticity')
print(f"   Cobb-Douglas: Q = {A} * L^{alpha} * K^{beta}")
print()
print("2. CARACTERÍSTICAS PRINCIPAIS:")
print(f"   - Retornos de escala: {returns_to_scale(production_cd)}")
print(f"   - Elasticidade do trabalho: {alpha}")
print(f"   - Elasticidade do capital: {beta}")
print(f"   - Soma das elasticidades: {alpha + beta} (retornos de escala)")
print()
print("3. RESULTADOS DE OTIMIZAÇÃO:")
print(f"   - Para produzir {target_output} unidades com w={wage_rate} e r={rental_rate}")
print(f"   - Trabalho ótimo: {L_optimal:.2f} unidades")
print(f"   - Capital ótimo: {K_optimal:.2f} unidades")
print(f"   - Custo mínimo: R$ {min_cost:.2f}")
print()
print("4. ANÁLISE DE EFICIÊNCIA:")
print(f"   - Eficiência média das empresas: {firms_df['efficiency'].mean():.1%}")
print(f"   - Variação de eficiência: {firms_df['efficiency'].min():.1%} a {firms_df['efficiency'].max():.1%}")
print(f"   - Potencial de melhoria: {(1 - firms_df['efficiency'].mean())*100:.1f} pontos percentuais")